In [1]:
import openai
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import logging
import os
import re
import requests
from bs4 import BeautifulSoup
from langchain.globals import set_debug, set_verbose

/home/albert/anaconda3/envs/langchain_env/lib/python3.10/site-packages/langchain_community/llms/__init__.py:412: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.llms.openai import OpenAI
/home/albert/anaconda3/envs/langchain_env/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


In [2]:
def get_budget_from_string(text):
    match = re.search(r'\d+', text.split('。')[0])
    return int(match.group()) if match else 30000

def summary_price(component_dict):
    total = 0
    for comp in component_dict.values():
        if comp and 'price' in comp and comp['price'].isdigit():
            total += int(comp['price'])
    return total

def component_prompt_maker(component_name):
    # 定義每個組件的專業知識
    component_info = {
        'Mother board': {
            'description': '主機板',
            'examples': 'ASUS ROG, MSI MEG, Gigabyte AORUS',
            'key_features': '晶片組、PCIe規格、記憶體支援、擴充插槽'
        },
        'Case': {
            'description': '機殼',
            'examples': 'Lian Li, Fractal Design, NZXT',
            'key_features': '散熱空間、擴充性、風扇安裝位置、硬碟位置'
        },
        'CPU': {
            'description': '處理器',
            'examples': 'Intel Core i9, AMD Ryzen 9',
            'key_features': '核心數、時脈速度、快取、功耗'
        },
        'GPU': {
            'description': '顯示卡',
            'examples': 'NVIDIA RTX 4090, RTX 4080, A5000',
            'key_features': '深度學習性能、CUDA核心數、顯存容量(建議>=12GB)、FP32/FP16效能'
            },
        'Memory': {
            'description': '記憶體',
            'examples': 'Corsair, G.SKILL, Crucial',
            'key_features': '容量、頻率、延遲、代數'
        },
        'Storage': {
            'description': '儲存裝置',
            'examples': 'Samsung SSD, WD Black, Seagate',
            'key_features': '容量、讀寫速度、耐用度、介面'
        },
        'Power': {
            'description': '電源供應器',
            'examples': 'Seasonic Prime 1000W, Corsair AX1200i',
            'key_features': '瓦數(深度學習建議>=1000W)、80 Plus認證、模組化、穩定度'
        },
        'Fan': {
            'description': '機殼風扇',
            'examples': 'Noctua NF-A12x25, Arctic P12, be quiet! Silent Wings',
            'key_features': '尺寸(通常是12cm或14cm)、風量、噪音、軸承類型、RGB燈效'
        }
    }

    info = component_info[component_name]
    prompt = f"""
    你是一個專業的電腦-{info['description']}銷售專家。
    你只能推薦 {component_name} 類別的產品，例如：{info['examples']}。
    評估產品時要考慮：{info['key_features']}。
    """


    
    prompt = prompt + """
    請根據以下資訊進行推薦：
    客戶的預算為：{budget}
    客戶的需求為：{require}
    目前已經有的電腦組合清單為：
    {component_list}

    請說明推薦原因，並提供產品名稱，回覆格式如下：
    [ 推薦原因 ] ： 耐用性強，可以使用到最新規格...
    [ 產品名稱 ] ： 產品型號
    [ 產品價格 ] ： 數字

    注意事項：
    1. 推薦原因的說明請限制在50個字以內
    2. 產品的價格請回覆一個完整的數字，中間不需要加入逗號
    3. 確保推薦的產品價格不超過給定的預算
    4. 盡量使用預算，不要推薦過低階的產品
    """

    return prompt

class Agent():
    def __init__(self, agent_name, prompt, model, require, budget, component_budget):
        self.agent_name = agent_name
        self.memory = []
        self.require = require
        self.budget = budget
        self.component_budget = component_budget  # 新增：儲存該組件的預算
        self.prompt_str = prompt  # 保存原始 prompt 字符串
        prompt = prompt + f"\n該組件的建議預算為：{component_budget} 元，請盡量在此預算內選擇適合的產品。"
        # 使用 pipe 風格串接組件
        template = ChatPromptTemplate.from_template(prompt)
        model = ChatOpenAI(model=model, temperature=0.7)
        parser = StrOutputParser()

        self.llm = template | model | parser
        
    def receive_message(self, messages):
        self.memory += messages
        
    def send_message(self, content, receive_obj):
        pass
    
    def action(self, component_dict):
        component_list = self.get_component_list(component_dict)
        
        # 準備輸入訊息
        user_message = {
            'component_list': component_list,
            'require': self.require,
            'budget': self.budget
        }
        
        # 使用 invoke 調用串接後的 llm
        message = self.llm.invoke(user_message)
        self.memory.append(message)
        parsed = self.parser_message(message)
        component_dict[self.agent_name] = parsed
        
        print(f"[Agent:{self.agent_name}] 初步推薦方案：{parsed}")
        return component_dict
        
    def revise_suggestion(self, component_dict, adjustment_reason):
        component_list = self.get_component_list(component_dict)
        current_price = int(self.parser_message(self.memory[-1])['price'])
        
        # 為修改建議創建新的串接
        revised_prompt = self.prompt_str + f"""
        目前協調者認為你的組件方案不符合要求，需要你再調整。
        原因：{adjustment_reason}

        你目前推薦的產品價格為 {current_price} 元，請推薦一個價格更低的產品。
        新推薦的產品價格必須低於目前的價格 {current_price} 元。

        請記住：
        1. 新推薦的產品必須比目前的便宜
        2. 儘量維持良好的性能
        3. 必須是同類型的產品
        4. 價格合理且符合市場行情
        """
        
        template = ChatPromptTemplate.from_template(revised_prompt)
        model = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0.2)  # 降低 temperature 使回應更穩定
        parser = StrOutputParser()
        
        chain = template | model | parser
        
        # 準備輸入訊息
        user_message = {
            'component_list': component_list,
            'require': self.require,
            'budget': self.budget
        }
        
        # 使用 invoke 調用
        message = chain.invoke(user_message)
        
        # 解析並驗證新推薦的價格是否確實更低
        new_recommendation = self.parser_message(message)
        if new_recommendation['price'] and int(new_recommendation['price']) >= current_price:
            print(f"[Warning] 新推薦的價格 ({new_recommendation['price']}) 不低於當前價格 ({current_price})，重試...")
            return self.revise_suggestion(component_dict, adjustment_reason)
        
        self.memory.append(message)
        component_dict[self.agent_name] = new_recommendation
        
        print(f"[Agent:{self.agent_name}] 調整後的新方案：{new_recommendation}")
        return component_dict
    
    def check_target(self, target):
        pass
    
    def end_stage(self):
        pass
    
    def get_component_list(self, component_dict):
        component_list = ''
        for i, (key, value) in enumerate(component_dict.items()):
            num = i + 1
            if value and 'price' in value and value['price']:
                component_list += f'{num}. {key}: {value["name"]} | price: {value["price"]}\n'
            else:
                component_list += f'{num}. {key}: None | price: None \n'
        return component_list
    
    def parser_message(self, message):
        pattern = {
            "reason": r"\[ 推薦原因 \] ：\s*(.*?)\s*\n",
            "name": r"\[ 產品名稱 \] ：\s*(.*?)\s*\n",
            "price": r"\[ 產品價格 \] ：\s*(\d+)"
        }

        results = {}
        for key, regex in pattern.items():
            match = re.search(regex, message)
            results[key] = match.group(1) if match else ""
        return results

class CoordinatorAgent:
    def __init__(self, require, budget, max_rounds=5):
        self.require = require
        self.budget = budget
        self.max_rounds = max_rounds
        self.agent_dict = {}
        
        # 定義標準組件名稱映射
        self.component_mapping = {
            'Mother board': ['Mother board', 'Motherboard', '主機板', 'MB'],
            'Case': ['Case', '機殼', '電腦機殼'],
            'CPU': ['CPU', '處理器'],
            'GPU': ['GPU', '顯示卡', 'Graphics Card'],
            'Memory': ['Memory', 'RAM', '記憶體'],
            'Device': ['Device', 'Storage', 'SSD', 'HDD', '硬碟'],
            'Power': ['Power', 'PSU', '電源供應器'],
            'Fan': ['Fan', 'Cooling', '散熱器', '風扇']
        }

    def _standardize_component_name(self, name):
        """將組件名稱標準化"""
        name = name.strip()
        for standard_name, variants in self.component_mapping.items():
            if name in variants or any(variant.lower() == name.lower() for variant in variants):
                return standard_name
        return name

    def allocate_budget(self, component_list):
        budget_prompt = f"""
        你是一個專業的電腦預算分配專家。請根據使用者的需求 "{self.require}" 和總預算 {self.budget} 元，分析並分配預算到各個組件。

        不同的電腦使用場景對組件的需求特性：
        1. GPU：遊戲、深度學習、影像處理等場景需求較高
        2. CPU：程式編譯、虛擬化、科學計算等場景需求較高
        3. Memory：多工處理、大數據分析、虛擬機等場景需求較高
        4. Storage：影片編輯、遊戲開發、大數據處理等場景需求較高

        請分析使用場景特性，並根據以下幾點進行預算分配：
        1. 評估哪些組件對此使用場景最關鍵
        2. 考慮組件間的效能平衡
        3. 確保關鍵組件有足夠的預算達到需求
        4. 根據當前市場行情做合理分配

        請嚴格按照以下格式回覆：

        【場景分析】
        使用場景：(描述使用場景特性)
        關鍵需求：(列出2-3個最關鍵的需求)

        【效能分析】
        最關鍵組件：(列出1-2個最關鍵的組件，並說明原因)
        次要組件：(列出2-3個次要組件，並說明原因)

        【組件相依性】
        1. (描述組件間的關鍵相依關係)
        2. (描述組件間的關鍵相依關係)

        【預算分配】
        {component_list[0]}：數字 (理由：限20字)
        {component_list[1]}：數字 (理由：限20字)
        {component_list[2]}：數字 (理由：限20字)
        {component_list[3]}：數字 (理由：限20字)
        {component_list[4]}：數字 (理由：限20字)
        {component_list[5]}：數字 (理由：限20字)
        {component_list[6]}：數字 (理由：限20字)
        {component_list[7]}：數字 (理由：限20字)

        規則：
        1. 數字必須是整數
        2. 所有數字總和必須等於 {self.budget}
        3. 只需要填寫數字，不需要加上元或百分比
        4. 必須嚴格按照上述格式回覆，保持組件名稱一致
        5. 預算分配必須考慮當前市場行情
        """

        template = ChatPromptTemplate.from_template(budget_prompt)
        model = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0.2)
        parser = StrOutputParser()
        chain = template | model | parser

        result = chain.invoke({})
        print (result)
        
        try:
            # 解析預算分配部分
            budget_section = result.split('【預算分配】')[1].strip()
            component_budgets = {}
            
            # 使用更嚴格的正則表達式來匹配
            pattern = r'([^：]+)：(\d+)\s*\(理由：[^)]+\)'
            matches = re.finditer(pattern, budget_section)
            
            for match in matches:
                comp_name = match.group(1).strip()
                amount = int(match.group(2))
                component_budgets[comp_name] = amount

            # 檢查是否所有組件都有預算
            for comp in component_list:
                if comp not in component_budgets:
                    raise KeyError(f"缺少組件預算：{comp}")

            # 驗證總預算
            total = sum(component_budgets.values())
            if total != self.budget:
                print(f"[Warning] 預算總和 ({total}) 不等於目標預算 ({self.budget})")
                # 進行調整...
                
            return component_budgets
            
        except Exception as e:
            print(f"[Error] LLM 預算分配解析失敗：{e}")
            return self._default_budget_allocation(component_list)



        def _default_budget_allocation(self, component_list):
            """預設的預算分配方式"""
            default_ratios = {
                'GPU': 0.35,
                'CPU': 0.20,
                'Mother board': 0.15,
                'Memory': 0.10,
                'Device': 0.08,
                'Power': 0.07,
                'Case': 0.03,
                'Fan': 0.02
            }
            
            component_budgets = {}
            remaining_budget = self.budget
            
            # 按比例分配預算
            for comp in component_list[:-1]:  # 除了最後一個組件
                ratio = default_ratios.get(comp, 0.1)
                amount = int(self.budget * ratio)
                component_budgets[comp] = amount
                remaining_budget -= amount
            
            # 最後一個組件拿到剩餘的預算，確保總和等於目標預算
            component_budgets[component_list[-1]] = remaining_budget
            
            return component_budgets



    def initialize_agents(self, component_list, model):
        """初始化所有代理人，並分配預算"""
        # 先分配預算
        component_budgets = self.allocate_budget(component_list)
        
        # 建立各組件的代理人
        for comp_name in component_list:
            prompt = component_prompt_maker(comp_name)
            agent = Agent(
                agent_name=comp_name,
                prompt=prompt,
                model=model,
                require=self.require,
                budget=self.budget,
                component_budget=component_budgets[comp_name]
            )
            self.agent_dict[comp_name] = agent
            
        return component_budgets

    def run(self):
        component_dict = {name: '' for name in self.agent_dict.keys()}
        print("[Coordinator] 開始取得所有組件的初步建議...")

        for agent_name, agent in self.agent_dict.items():
            component_dict = agent.action(component_dict)

        rounds = 0
        while rounds < self.max_rounds:
            rounds += 1
            total_price = summary_price(component_dict)
            print(f"\n[Coordinator] 第{rounds}回合檢查總價：{total_price} (預算：{self.budget})")

            if total_price <= self.budget:
                print("[Coordinator] 總價在預算內，討論結束。")
                break
            else:
                print("[Coordinator] 總價超出預算，嘗試要求代理人調整...")
                adjustment_reason = f"總價格 {total_price} 超出預算 {self.budget}，請降低價格。"

                # 找出最超出其建議預算的組件
                sorted_components = sorted(
                    [(name, info) for name, info in component_dict.items() if info and info['price'].isdigit()],
                    key=lambda x: int(x[1]['price']) - self.agent_dict[x[0]].component_budget,
                    reverse=True
                )
                
                if sorted_components:
                    comp_name, comp_value = sorted_components[0]
                    print(f"[Coordinator] 請 {comp_name} 降低價格以符合預算...")
                    component_dict = self.agent_dict[comp_name].revise_suggestion(component_dict, adjustment_reason)

        return component_dict





In [3]:
require = '資工博士生想組一台 100000元的 Deep learning 主機，機殼要有 RGB 電競風格。'
budget = get_budget_from_string(require)

# 定義需要的組件列表
component_list = ['Mother board', 'Case', 'CPU', 'GPU', 'Memory', 'Storage', 'Power', 'Fan']



# 創建協調者並初始化所有代理人
coordinator = CoordinatorAgent(require=require, budget=budget, max_rounds=5)

# 初始化所有代理人並獲取預算分配
component_budgets = coordinator.initialize_agents(
    component_list=component_list,
    model="gpt-4o-2024-11-20"
)

# 顯示初始預算分配
print("\n[Coordinator] 初始預算分配：")
for comp, comp_budget in component_budgets.items():
    print(f"{comp}: {comp_budget} 元")

# 執行協調過程
final_component_dict = coordinator.run()

# 輸出最終結果
print("\n[Coordinator] 最終組合清單：")
for k, v in final_component_dict.items():
    print(f"{k}: {v}")
print(f"最終總價格：{summary_price(final_component_dict)}")

【場景分析】  
使用場景：深度學習主機，需高效處理大量矩陣運算與模型訓練，並兼具電競風格外觀。  
關鍵需求：高效GPU運算性能、大容量記憶體、穩定供電支持。  

【效能分析】  
最關鍵組件：  
1. GPU：深度學習依賴GPU進行大規模矩陣運算，性能直接影響訓練速度。  
2. Memory：深度學習模型需大量記憶體以處理數據與模型參數。  

次要組件：  
1. CPU：需支持多核運算以協助數據預處理與模型部署。  
2. Power：穩定供電對高功耗GPU與其他組件至關重要。  
3. Storage：需快速存取數據，建議使用NVMe SSD。  

【組件相依性】  
1. GPU與Power：高效GPU需穩定且足夠的電力支持。  
2. CPU與Memory：CPU性能需與記憶體容量匹配，避免瓶頸。  

【預算分配】  
Mother board：10000 (理由：需支持高效GPU與多記憶體插槽)  
Case：5000 (理由：RGB電競風格，兼具散熱與空間)  
CPU：15000 (理由：需多核高效處理數據預處理)  
GPU：40000 (理由：深度學習核心，需高效運算性能)  
Memory：15000 (理由：大容量記憶體支持模型訓練)  
Storage：10000 (理由：快速存取數據，建議NVMe SSD)  
Power：8000 (理由：穩定供電，支持高功耗組件)  
Fan：7000 (理由：高效散熱，保證穩定運行)  
[Warning] 預算總和 (110000) 不等於目標預算 (100000)

[Coordinator] 初始預算分配：
Mother board: 10000 元
Case: 5000 元
CPU: 15000 元
GPU: 40000 元
Memory: 15000 元
Storage: 10000 元
Power: 8000 元
Fan: 7000 元
[Coordinator] 開始取得所有組件的初步建議...
[Agent:Mother board] 初步推薦方案：{'reason': '支援最新PCIe 5.0與DDR5記憶體，擴充性佳，適合AI運算', 'name': 'ASUS ROG STRIX Z790-E GAMING WIFI', 'price': '9850'}
[Ag